In [39]:
import numpy as np
import matplotlib.pyplot as plt

In [40]:
class PottsModel:
#Clase del modelo, dá las condiciones iniciales de la grilla, inicializa los spines.
    def __init__(self, L, q, T, J=1.0):
        self.L = L #dimensión de la grilla en m
        self.q = q #Cantidad de estados
        self.T = T #Temperatura J/k_b
        self.J = J #Constante de acoplamiento J
        self.spins = None
        self.initialize_spins()


    def initialize_spins(self):
        # Inicializar spines aleatoriamente
        self.spins = np.random.randint(0, self.q, (self.L, self.L))
        for i in self.spins:
            self.spins[i] = 1

    def local_energy(self, i, j):
        # Energía en sitio (i,j)
        beginning_spin = self.spins[i,j]
        energy = 0
        for di, dj in [(1,0), (-1,0), (0,1), (0,-1)]:
            ni, nj = (i + di) % self.L, (j + dj) % self.L #rutina para condiciones periódicas.
            if beginning_spin == self.spins[ni,nj]:
                energy -= self.J
        return energy

    def total_energy(self):
    # Energía total del sistema
        energy = 0
        for i in range(self.L):
            for j in range(self.L):
                #Enlace horizontal
                if self.spins[i, j] == self.spins[i, (j+1) % self.L]:
                    energy -= self.J

                # Enlace VERTICAL:
                if self.spins[i, j] == self.spins[(i+1) % self.L, j]:
                    energy -= self.J

        return energy
    def magnetization(self):
      if self.q == 1:
            return 1.0

      elif self.q == 2:
        # ISING: con signo
        spins_ising = 2 * self.spins.astype(float) - 1
        return np.mean(spins_ising)

      else:
        # POTTS: parámetro de orden (sin signo)
        counts = np.bincount(self.spins.flatten(), minlength=self.q)
        max_fraction = np.max(counts) / self.spins.size
        return (max_fraction - 1/self.q) / (1 - 1/self.q)


In [41]:
class MonteCarlo:
    def __init__(self, model):
      ## definir tamano de listas
        self.model = model
        self.energy_history = []
        self.mag_history = []

    def metropolis_step(self):
        # Un paso del algoritmo de Metropolis
        L = self.model.L
        i, j = np.random.randint(0, L, 2)
        old_spin = self.model.spins[i, j]
        old_energy = self.model.local_energy(i, j)

        # Proponer nuevo spin
        new_spin = np.random.randint(0, self.model.q)
        self.model.spins[i, j] = new_spin
        new_energy = self.model.local_energy(i, j)

        # Criterio de Metropolis
        # Comparar con la distribución de Boltzman
        delta_E = new_energy - old_energy
        if delta_E > 0 and np.random.random() >= np.exp(-delta_E / self.model.T):
            self.model.spins[i, j] = old_spin  # Rechazar
        # Aceptar implícitamente si no se rechaza

    def run(self, steps, thermalization):
        # Ejecutar simulación
        for step in range(steps + thermalization):
            for _ in range(self.model.L * self.model.L):
                self.metropolis_step()
            #Steps de termalización para que la simulación se estabilize.
            #Las cantidades físicas se calculan con los steps que siguen despues de termalización
            if step >= thermalization:
                E = self.model.total_energy()
                self.energy_history.append(E)
                self.mag_history.append(self.model.magnetization())


In [56]:
class Analysis:
    def __init__(self, mc_data):
        self.mc = mc_data
        self.energy = mc_data.energy_history
        self.magnetization = mc_data.mag_history

    def specific_heat(self):
        # Calor específico
        E = np.array(self.energy)

        return (np.var(E) / ((self.mc.model.L**2) *self.mc.model.T**2))

    def susceptibility(self):
        #Susceptibilidad magnética
        M = np.array(self.magnetization)
        return (np.var(M) /((self.mc.model.L**2)*self.mc.model.T))

In [57]:
Potts = PottsModel( L = 30, q = 2,T = 2.269)

mc_data = MonteCarlo(Potts)
mc_data.run(steps = 1000, thermalization = 1000)

In [58]:
results = Analysis(mc_data)


In [59]:

print(results.specific_heat())
print(results.susceptibility())

0.11535994206052501
1.583815411513565e-06


In [46]:
np.mean(results.energy)


np.float64(-1118.39)

In [47]:
np.mean(results.magnetization)

np.float64(0.0032755555555555563)

In [48]:

print("=== ANÁLISIS COMPLETO ===")
print(f"Energía promedio: {np.mean(results.energy):.4f}")
print(f"Magnetización promedio: {np.mean(results.magnetization):.6f}")
print(f"Varianza de energía: {np.var(results.energy):.6f}")
print(f"Varianza de magnetización: {np.var(results.magnetization):.8f}")
print(f"Valores de magnetización (primeros 10): {results.magnetization[:10]}")

=== ANÁLISIS COMPLETO ===
Energía promedio: -1118.3900
Magnetización promedio: 0.003276
Varianza de energía: 545.635900
Varianza de magnetización: 0.00327777
Valores de magnetización (primeros 10): [np.float64(0.04888888888888889), np.float64(0.1), np.float64(0.06444444444444444), np.float64(0.06666666666666667), np.float64(0.09555555555555556), np.float64(0.12), np.float64(0.07111111111111111), np.float64(0.08), np.float64(0.09777777777777778), np.float64(0.03333333333333333)]
